# Keyword Search

## Setup

Load needed API keys and relevant Python libaries.

In [1]:
# !pip install cohere
# !pip install weaviate-client

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

위키백과 데이터베이스에 액세스하기 위해 Weaviate를 임포트하는 것부터 시작해 보겠습니다.

In [3]:
import weaviate
# auth_config = weaviate.auth.AuthApiKey(api_key=os.environ['WEAVIATE_API_KEY'])
auth_config = weaviate.auth.AuthApiKey(api_key='76320a90-53d8-42bc-b41d-678647c6672e')

In [4]:
client = weaviate.Client(
    url='https://cohere-demo.weaviate.network',
    # url=os.environ['WEAVIATE_API_URL'],
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.environ['COHERE_API_KEY'],
    }
)

In [5]:
client.is_ready() 

True

# Keyword Search

In [6]:
def keyword_search(query,
                   results_lang='en',
                   properties = ["title","url","text"],
                   num_results=3):

    where_filter = {
    "path": ["lang"],
    "operator": "Equal",
    "valueString": results_lang
    }
    
    response = (
        client.query.get("Articles", properties)
        .with_bm25(
            query=query
        )
        .with_where(where_filter)
        .with_limit(num_results)
        .do()
        )

    result = response['data']['Get']['Articles']
    return result

In [7]:
query = "What is the most viewed televised event?"
keyword_search_results = keyword_search(query)
print(keyword_search_results)

[{'text': 'The most active Gamergate supporters or "Gamergaters" said that Gamergate was a movement for ethics in games journalism, for protecting the "gamer" identity, and for opposing "political correctness" in video games and that any harassment of women was done by others not affiliated with Gamergate. They argued that the close relationships between journalists and developers demonstrated a conspiracy among reviewers to focus on progressive social issues. Some supporters pointed to what they considered disproportionate praise for games such as "Depression Quest" and "Gone Home", which feature unconventional gameplay and stories with social implications, while they viewed traditional AAA games as downplayed. False claims of the "ethics in game journalism" had started as early as 2012, when Geoff Keighley was accused of such unethical behavior when he was presenting information about "Halo 4" among advertisements for Mountain Dew and Doritos, an event called "Doritosgate" in the gam

### 검색 옵션 수정 시도
- Other languages to try: `en, de, fr, es, it, ja, ar, zh, ko, hi`

In [8]:
properties = ["text", "title", "url", 
             "views", "lang"]

In [9]:
def print_result(result):
    """ Print results with colorful formatting """
    for i,item in enumerate(result):
        print(f'item {i}')
        for key in item.keys():
            print(f"{key}:{item.get(key)}")
            print()
        print()

In [10]:
print_result(keyword_search_results)

item 0
text:The most active Gamergate supporters or "Gamergaters" said that Gamergate was a movement for ethics in games journalism, for protecting the "gamer" identity, and for opposing "political correctness" in video games and that any harassment of women was done by others not affiliated with Gamergate. They argued that the close relationships between journalists and developers demonstrated a conspiracy among reviewers to focus on progressive social issues. Some supporters pointed to what they considered disproportionate praise for games such as "Depression Quest" and "Gone Home", which feature unconventional gameplay and stories with social implications, while they viewed traditional AAA games as downplayed. False claims of the "ethics in game journalism" had started as early as 2012, when Geoff Keighley was accused of such unethical behavior when he was presenting information about "Halo 4" among advertisements for Mountain Dew and Doritos, an event called "Doritosgate" in the ga

In [13]:
# query = "What is the most viewed televised event?"
query = "한국 영화 흥행 기록"
keyword_search_results = keyword_search(query, results_lang='ko')
print_result(keyword_search_results)

item 0
text:다음은 영화의 흥행 순위를 알 수 있는 대한민국에서 개봉된 영화의 총 입장 관객 순위이다. 영화 흥행 기준을 영화의 총 수입을 기준으로 삼는 것과 총 관객 수를 기준으로 삼는 것이 있는데, 보통 미국은 그 기준이 총 수입이며, 대한민국은 총 관객 수를 그 기준으로 삼아 집계하고 있다.

title:대한민국의 영화 흥행 기록

url:https://ko.wikipedia.org/wiki?curid=33145


item 1
text:다음은 영화의 흥행 순위를 알 수 있는 북미에서 개봉된 영화의 총 수입 순위이다. 영화 흥행 기준을 영화의 총 수입을 기준으로 삼는 것과 총 관객 수를 기준으로 삼는 것이 있는데, 보통 미국은 그 기준이 총 수입이며, 대한민국은 총 관객 수를 그 기준으로 삼아 집계하고 있다.

title:미국과 캐나다의 영화 흥행 기록

url:https://ko.wikipedia.org/wiki?curid=1137898


item 2
text:다음은 영화의 흥행 순위를 알 수 있는 영국에서 개봉된 영화의 총 수입 순위이다. 영화 흥행 기준을 영화의 총 수입을 기준으로 삼는 것과 총 관객 수를 기준으로 삼는 것이 있는데, 보통 영국은 그 기준이 총 수입은 극장 상영 수익만을 기준으로 하며, 홈 비디오나 텔레비전 방영 등의 수익은 포함하지 않는다.

title:영국의 영화 흥행 기록

url:https://ko.wikipedia.org/wiki?curid=2888654




In [12]:
query = "What is the most viewed televised event?"
keyword_search_results = keyword_search(query, results_lang='de')
print_result(keyword_search_results)

item 0
text:Das nächste Album "Extinction Level Event: The Final World Front", dessen Titel vom Film "Deep Impact" inspiriert war, enthielt die Single "Gimme Some More", die im Januar 1999 Platz 5 der britischen Charts belegte. Mit Janet Jackson sang er außerdem den Erfolgshit "What’s It Gonna Be?!"

title:Busta Rhymes

url:https://de.wikipedia.org/wiki?curid=238582


item 1
text:Der Begriff Strukturalismus ist vom Wort Struktur abgeleitet und wurde vereinzelt seit Mitte des 19. Jahrhunderts verwendet. In der Psychologie wurde er 1907 von James Rowland Angell in einem Aufsatz über Edward Bradford Titchener eingeführt. In der Sprach- und Literaturwissenschaft wurde er seit den 1920er vom Prager linguistischen Kreis verwendet. Dort knüpfte Roman Jakobson an die Schule des Russischen Formalismus an und verwendete den Begriff „Strukturalismus“ erstmals 1929 in einem in der Prager Zeitschrift "Čin" auf Tschechisch unter dem Titel "Romantischer Panslawismus – neue Slawistik" ("Romantické vše

## How to get your own API key

For this course, an API key is provided for you.  If you would like to develop projects with Cohere's API outside of this classroom, you can register for an API key [here](https://dashboard.cohere.ai/welcome/register?utm_source=partner&utm_medium=website&utm_campaign=DeeplearningAI).